In [97]:
# imports
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')   
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression   
from sklearn.metrics import mean_squared_error, r2_score    
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso   



In [127]:
# Loading dataset: power_multi_household_daily.csv
power_path = "power_multi_household_daily.csv"

df_power_daily = pd.read_csv(power_path, low_memory=False)
df_power_daily

,meter_id,date,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean
0,MTR_001,2006-12-16,2.617495,0.882098,0.222336,6.660460,236.243763,2.922896,13.082828
1,MTR_001,2006-12-17,2.018833,1.032528,0.097898,6.024453,240.087028,4.051467,9.999028
2,MTR_001,2006-12-18,1.311082,0.862724,0.065486,5.258530,241.231694,3.719576,6.421667
3,MTR_001,2006-12-19,0.992268,1.063872,0.020072,6.750390,241.999313,3.069492,4.926389
4,MTR_001,2006-12-20,1.325185,1.134982,0.026983,5.161571,242.308063,3.345704,6.467361
...,...,...,...,...,...,...,...,...,...
14415,MTR_010,2010-11-22,1.624360,1.346636,0.146076,6.865426,240.197118,3.916832,6.014167
14416,MTR_010,2010-11-23,1.254788,1.004583,0.094465,5.337459,240.575528,3.833612,4.674444
14417,MTR_010,2010-11-24,1.431164,1.020773,0.133426,6.534909,240.030965,3.747974,5.249583
14418,MTR_010,2010-11-25,1.139526,0.955009,0.156308,5.809185,241.536257,2.714365,4.169444


In [99]:
# parse date and set index if present
if 'date' in df_power_daily.columns:
    df_power_daily['date'] = pd.to_datetime(df_power_daily['date'], errors='coerce')
    df_power_daily.set_index('date', inplace=True)


In [100]:
# basic preview and diagnostics
print(f"Loaded '{power_path}' — shape: {df_power_daily.shape}")
print('\nColumn dtypes:')
print(df_power_daily.dtypes)
print('\nMissing values per column:')
print(df_power_daily.isnull().sum())


Loaded 'power_multi_household_daily.csv' — shape: (14420, 8)

Column dtypes:
meter_id             object
daily_mean_power    float64
daily_std_power     float64
daily_min_power     float64
daily_max_power     float64
voltage_mean        float64
voltage_std         float64
intensity_mean      float64
dtype: object

Missing values per column:
meter_id             0
daily_mean_power    90
daily_std_power     90
daily_min_power     90
daily_max_power     90
voltage_mean        90
voltage_std         90
intensity_mean      90
dtype: int64


In [136]:
# checking missing values
print('\nMissing values per column:')
print(df_power_daily.isnull().sum())


Missing values per column:
meter_id            0
date                0
daily_mean_power    0
daily_std_power     0
daily_min_power     0
daily_max_power     0
voltage_mean        0
voltage_std         0
intensity_mean      0
dtype: int64


In [101]:
# basic dataset-specific checks
if 'meter_id' in df_power_daily.columns:
    print('\nUnique meters:', df_power_daily['meter_id'].nunique())
for col in ['daily_consumption','consumption','Global_active_power','energy_kwh','total_kwh']:
    if col in df_power_daily.columns:
        print(f"\nSummary for '{col}':")
        display(df_power_daily[col].describe())


Unique meters: 10


In [126]:
# Loading dataset: kplc_daily_schedule.csv 
# file path
kplc_path = "kplc_daily_schedule.csv"

# load dataset
df_kplc = pd.read_csv(kplc_path, low_memory=False)
df_kplc


,date,scheduled_outage_today,n_scheduled_events,total_scheduled_minutes
0,2026-01-18,1,3,1380.0
1,2026-01-20,1,4,1950.0
2,2026-01-21,1,2,960.0
3,2026-01-22,1,3,1380.0
4,2026-01-23,1,1,450.0


In [103]:
# loading dataset: nairobi_weather_2007_2008.csv
# file path
weather_path = "nairobi_weather_2007_2008.csv"

# load dataset
df_weather = pd.read_csv(weather_path, low_memory=False)
df_weather.head()


,date,tmax,tmin,prcp,wspd_max
0,2007-01-01,24.7,13.1,1.0,17.6
1,2007-01-02,23.2,13.7,0.1,17.4
2,2007-01-03,22.6,14.8,0.9,20.0
3,2007-01-04,21.5,14.9,2.3,21.4
4,2007-01-05,23.3,12.9,0.0,18.5


In [104]:
# parse any `date` column to datetime (safe conversion)
for df, name in [(df_kplc, 'df_kplc'), (df_weather, 'df_weather')]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [105]:
# quick previews and summaries
print(f"Loaded '{kplc_path}' — shape: {df_kplc.shape}")
print('\nColumn dtypes:')
print(df_kplc.dtypes)
print('\nMissing values per column:')
print(df_kplc.isnull().sum())
if 'date' in df_kplc.columns:
    print(f"{kplc_path} date range: {df_kplc['date'].min()} — {df_kplc['date'].max()}")

print('\n' + '-'*60 + '\n')

print(f"Loaded '{weather_path}' — shape: {df_weather.shape}")
print('\nColumn dtypes:')
print(df_weather.dtypes)
print('\nMissing values per column:')
print(df_weather.isnull().sum())
if 'date' in df_weather.columns:
    print(f"{weather_path} date range: {df_weather['date'].min()} — {df_weather['date'].max()}")

Loaded 'kplc_daily_schedule.csv' — shape: (5, 4)

Column dtypes:
date                       datetime64[ns]
scheduled_outage_today              int64
n_scheduled_events                  int64
total_scheduled_minutes           float64
dtype: object

Missing values per column:
date                       0
scheduled_outage_today     0
n_scheduled_events         0
total_scheduled_minutes    0
dtype: int64
kplc_daily_schedule.csv date range: 2026-01-18 00:00:00 — 2026-01-23 00:00:00

------------------------------------------------------------

Loaded 'nairobi_weather_2007_2008.csv' — shape: (731, 5)

Column dtypes:
date        datetime64[ns]
tmax               float64
tmin               float64
prcp               float64
wspd_max           float64
dtype: object

Missing values per column:
date        0
tmax        0
tmin        0
prcp        0
wspd_max    0
dtype: int64
nairobi_weather_2007_2008.csv date range: 2007-01-01 00:00:00 — 2008-12-31 00:00:00


In [106]:
# make copies
df_kplc_clean = df_kplc.copy()
df_weather_clean = df_weather.copy()

In [107]:
# df_kplc cleaning
print('df_kplc: original shape ->', df_kplc_clean.shape)


df_kplc: original shape -> (5, 4)


In [108]:
# drop exact duplicate rows
dup_k = df_kplc_clean.duplicated().sum()
if dup_k:
    df_kplc_clean = df_kplc_clean.drop_duplicates()
    print(f'dropped {dup_k} duplicate rows from df_kplc')

In [109]:
# ensure numeric types
num_cols_k = ['scheduled_outage_today', 'n_scheduled_events', 'total_scheduled_minutes']
for c in num_cols_k:
    if c in df_kplc_clean.columns:
        df_kplc_clean[c] = pd.to_numeric(df_kplc_clean[c], errors='coerce')

In [110]:
# create date features
if 'date' in df_kplc_clean.columns and pd.api.types.is_datetime64_any_dtype(df_kplc_clean['date']):
    df_kplc_clean = df_kplc_clean.sort_values('date')
    df_kplc_clean['year'] = df_kplc_clean['date'].dt.year
    df_kplc_clean['month'] = df_kplc_clean['date'].dt.month
    df_kplc_clean['day'] = df_kplc_clean['date'].dt.day
    df_kplc_clean['weekday'] = df_kplc_clean['date'].dt.weekday
    df_kplc_clean['is_weekend'] = df_kplc_clean['weekday'] >= 5
    df_kplc_clean.set_index('date', inplace=True)


In [111]:
# outlier flag for total minutes (IQR)
if 'total_scheduled_minutes' in df_kplc_clean.columns:
    q1 = df_kplc_clean['total_scheduled_minutes'].quantile(0.25)
    q3 = df_kplc_clean['total_scheduled_minutes'].quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    df_kplc_clean['total_minutes_outlier'] = ((df_kplc_clean['total_scheduled_minutes'] < lower) | (df_kplc_clean['total_scheduled_minutes'] > upper))

In [112]:
# simple imputation placeholder (only apply if missing exists)
if df_kplc_clean.isnull().sum().sum() > 0:
    df_kplc_clean['total_scheduled_minutes'].fillna(df_kplc_clean['total_scheduled_minutes'].median(), inplace=True)

print('df_kplc_clean shape ->', df_kplc_clean.shape)
print('-'*60)


df_kplc_clean shape -> (5, 9)
------------------------------------------------------------


In [113]:
# df_weather cleaning 
df_weather_clean.shape


(731, 5)

In [114]:
# drop exact duplicate rows
dup_w = df_weather_clean.duplicated().sum()
if dup_w:
    df_weather_clean = df_weather_clean.drop_duplicates()
    print(f'dropped {dup_w} duplicate rows from df_weather')

In [115]:
# ensure numeric types for weather columns
for c in ['tmax', 'tmin', 'prcp', 'wspd_max']:
    if c in df_weather_clean.columns:
        df_weather_clean[c] = pd.to_numeric(df_weather_clean[c], errors='coerce')

In [116]:
# date features and index
if 'date' in df_weather_clean.columns and pd.api.types.is_datetime64_any_dtype(df_weather_clean['date']):
    df_weather_clean = df_weather_clean.sort_values('date')
    df_weather_clean['temp_range'] = df_weather_clean['tmax'] - df_weather_clean['tmin']
    df_weather_clean['precip_flag'] = df_weather_clean['prcp'] > 0
    df_weather_clean['year'] = df_weather_clean['date'].dt.year
    df_weather_clean['month'] = df_weather_clean['date'].dt.month
    df_weather_clean.set_index('date', inplace=True)

In [117]:
# forward/backward fill or interpolation if missing
if df_weather_clean.isnull().sum().sum() > 0:
    df_weather_clean = df_weather_clean.sort_index().interpolate(method='time').ffill().bfill()

print('df_weather_clean shape ->', df_weather_clean.shape)



df_weather_clean shape -> (731, 8)


In [118]:
# Attempting merge on date 
df_merged = pd.merge(df_kplc_clean.reset_index(), df_weather_clean.reset_index(), on='date', how='left', suffixes=('_kplc','_weather'))
print('Merged shape:', df_merged.shape)

Merged shape: (5, 18)


In [119]:
# counting how many rows have matching weather
matched = df_merged['tmax'].notna().sum() if 'tmax' in df_merged.columns else 0
print('Rows with matching weather data:', matched)

Rows with matching weather data: 0


In [120]:
# checking date-range overlap
k_min, k_max = df_kplc_clean.index.min(), df_kplc_clean.index.max()
w_min, w_max = df_weather_clean.index.min(), df_weather_clean.index.max()
print(f"kplc date range: {k_min} — {k_max}")
print(f"weather date range: {w_min} — {w_max}")

kplc date range: 2026-01-18 00:00:00 — 2026-01-23 00:00:00
weather date range: 2007-01-01 00:00:00 — 2008-12-31 00:00:00


In [121]:
# quick preview of cleaned / merged data
print('\nSample (df_kplc_clean):')
print('\nSample (df_weather_clean):')
print('\nSample (df_merged):')
display(df_merged.head())


Sample (df_kplc_clean):

Sample (df_weather_clean):

Sample (df_merged):



Sample (df_kplc_clean):

Sample (df_weather_clean):

Sample (df_merged):


,date,scheduled_outage_today,n_scheduled_events,total_scheduled_minutes,year_kplc,month_kplc,day,weekday,is_weekend,total_minutes_outlier,tmax,tmin,prcp,wspd_max,temp_range,precip_flag,year_weather,month_weather
0,2026-01-18,1,3,1380.0,2026,1,18,6,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2026-01-20,1,4,1950.0,2026,1,20,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-21,1,2,960.0,2026,1,21,2,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-22,1,3,1380.0,2026,1,22,3,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-01-23,1,1,450.0,2026,1,23,4,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# Info check for the 3 main datasets
print("=" * 80)
print("Dataset 1: df_power_daily")
print("=" * 80)
df_power_daily.info()

Dataset 1: df_power_daily
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14420 entries, 2006-12-16 to 2010-11-26
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   meter_id          14420 non-null  object 
 1   daily_mean_power  14330 non-null  float64
 2   daily_std_power   14330 non-null  float64
 3   daily_min_power   14330 non-null  float64
 4   daily_max_power   14330 non-null  float64
 5   voltage_mean      14330 non-null  float64
 6   voltage_std       14330 non-null  float64
 7   intensity_mean    14330 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1013.9+ KB


In [123]:
print("\n" + "=" * 80)
print("Dataset 2: df_kplc")
print("=" * 80)
df_kplc.info()


Dataset 2: df_kplc
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     5 non-null      datetime64[ns]
 1   scheduled_outage_today   5 non-null      int64         
 2   n_scheduled_events       5 non-null      int64         
 3   total_scheduled_minutes  5 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 292.0 bytes


In [124]:
print("\n" + "=" * 80)
print("Dataset 3: df_weather")
print("=" * 80)
df_weather.info()


Dataset 3: df_weather
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      731 non-null    datetime64[ns]
 1   tmax      731 non-null    float64       
 2   tmin      731 non-null    float64       
 3   prcp      731 non-null    float64       
 4   wspd_max  731 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 28.7 KB


In [129]:
# --- Merge: Power Multi-Household Daily + Nairobi Weather
print("Merging df_power_daily and df_weather on date...")

# Reset power index to make date a column for merging
power_reset = df_power_daily.reset_index()

# Ensure both date columns are datetime
power_reset['date'] = pd.to_datetime(power_reset['date'], errors='coerce')
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')

# Merge on date
df_power_weather = pd.merge(
    power_reset,
    df_weather,
    on='date',
    how='inner'  # inner join to keep only overlapping dates
)

print(f"\nMerged dataset 'df_power_weather':")
print(f"  Shape: {df_power_weather.shape}")
print(f"  Date range: {df_power_weather['date'].min()} → {df_power_weather['date'].max()}")

print(f"\nColumns ({df_power_weather.shape[1]}):")
print(df_power_weather.columns.tolist())

print(f"\nMissing values:")
print(df_power_weather.isnull().sum())

print(f"\nPreview:")
display(df_power_weather.head())

Merging df_power_daily and df_weather on date...

Merged dataset 'df_power_weather':
  Shape: (7310, 14)
  Date range: 2007-01-01 00:00:00 → 2008-12-31 00:00:00

Columns (14):
['index', 'meter_id', 'date', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max']

Missing values:
index                0
meter_id             0
date                 0
daily_mean_power    10
daily_std_power     10
daily_min_power     10
daily_max_power     10
voltage_mean        10
voltage_std         10
intensity_mean      10
tmax                 0
tmin                 0
prcp                 0
wspd_max             0
dtype: int64

Preview:


,index,meter_id,date,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean,tmax,tmin,prcp,wspd_max
0,16,MTR_001,2007-01-01,1.634586,0.782358,0.086498,3.066800,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
1,1458,MTR_002,2007-01-01,2.388304,1.142670,0.202939,4.471271,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
2,2900,MTR_003,2007-01-01,1.312100,0.688413,0.041358,2.655460,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
3,4342,MTR_004,2007-01-01,2.122846,1.016589,0.214765,3.969411,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
4,5784,MTR_005,2007-01-01,1.976886,0.947051,0.170827,3.748946,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6


In [146]:
# Analysis: Missing values in df_power_daily (meter_id 14420 vs others 14330

# Identify rows with meter_id but missing power metrics
power_cols = ['daily_mean_power', 'daily_std_power', 'daily_min_power', 
              'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean']

In [147]:
# Rows where ANY power column is null
incomplete_rows = df_power_daily[df_power_daily[power_cols].isnull().any(axis=1)]

print(f"\nRows with meter_id but missing power metrics: {len(incomplete_rows)}")
print(f"  Total rows: {len(df_power_daily)}")
print(f"  Complete rows: {len(df_power_daily) - len(incomplete_rows)}")
print(f"  Incomplete rows (missing metrics): {len(incomplete_rows)}")


Rows with meter_id but missing power metrics: 0
  Total rows: 14330
  Complete rows: 14330
  Incomplete rows (missing metrics): 0


In [ ]:
#  Drop incomplete rows from df_power_daily
print("Dropping rows with missing power metrics...")

power_cols = ['daily_mean_power', 'daily_std_power', 'daily_min_power', 
              'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean']

# Before
shape_before = df_power_daily.shape
print(f"\nBefore: {shape_before}")

# Dropping rows where ANY power column is null
df_power_daily = df_power_daily.dropna(subset=power_cols, how='any')

# After
shape_after = df_power_daily.shape
print(f"After:  {shape_after}")
print(f"Rows removed: {shape_before[0] - shape_after[0]}")

# Verify no missing values in power columns
print(f"\nMissing values in power columns (post-drop):")
print(df_power_daily[power_cols].isnull().sum())

print(f"\ndf_power_daily is now clean and ready for analysis.")
print(f"Unique meters: {df_power_daily['meter_id'].nunique()}")
print(f"Date range: {df_power_daily.index.min()} → {df_power_daily.index.max()}")

Dropping rows with missing power metrics...

Before: (14330, 9)
After:  (14330, 9)
Rows removed: 0

Missing values in power columns (post-drop):
daily_mean_power    0
daily_std_power     0
daily_min_power     0
daily_max_power     0
voltage_mean        0
voltage_std         0
intensity_mean      0
dtype: int64

df_power_daily is now clean and ready for analysis.
Unique meters: 10
Date range: 0 → 14419


In [142]:
# --- Merge CLEANED: Power Multi-Household Daily + Nairobi Weather
print("Merging cleaned df_power_daily with df_weather...")

# Reset power index to make date a column for merging
power_reset = df_power_daily.reset_index()

# Ensure both date columns are datetime
power_reset['date'] = pd.to_datetime(power_reset['date'], errors='coerce')
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')

# Merge on date (inner join keeps only overlapping dates)
df_power_weather_clean = pd.merge(
    power_reset,
    df_weather,
    on='date',
    how='inner'
)

print(f"\n✓ Merged dataset 'df_power_weather_clean':")
print(f"  Shape: {df_power_weather_clean.shape}")
print(f"  Date range: {df_power_weather_clean['date'].min()} → {df_power_weather_clean['date'].max()}")
print(f"  Columns: {df_power_weather_clean.shape[1]}")
print(f"  Unique meters: {df_power_weather_clean['meter_id'].nunique()}")
print(f"  Missing values: {df_power_weather_clean.isnull().sum().sum()}")

print(f"\nColumn list:")
print(df_power_weather_clean.columns.tolist())

print(f"\nPreview ():")
display(df_power_weather_clean)

Merging cleaned df_power_daily with df_weather...

✓ Merged dataset 'df_power_weather_clean':
  Shape: (7300, 14)
  Date range: 2007-01-01 00:00:00 → 2008-12-31 00:00:00
  Columns: 14
  Unique meters: 10
  Missing values: 0

Column list:
['index', 'meter_id', 'date', 'daily_mean_power', 'daily_std_power', 'daily_min_power', 'daily_max_power', 'voltage_mean', 'voltage_std', 'intensity_mean', 'tmax', 'tmin', 'prcp', 'wspd_max']

Preview ():


,index,meter_id,date,daily_mean_power,daily_std_power,daily_min_power,daily_max_power,voltage_mean,voltage_std,intensity_mean,tmax,tmin,prcp,wspd_max
0,16,MTR_001,2007-01-01,1.634586,0.782358,0.086498,3.066800,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
1,1458,MTR_002,2007-01-01,2.388304,1.142670,0.202939,4.471271,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
2,2900,MTR_003,2007-01-01,1.312100,0.688413,0.041358,2.655460,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
3,4342,MTR_004,2007-01-01,2.122846,1.016589,0.214765,3.969411,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
4,5784,MTR_005,2007-01-01,1.976886,0.947051,0.170827,3.748946,240.128979,2.216851,7.916944,24.7,13.1,1.0,17.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7295,7956,MTR_006,2008-12-31,1.025914,0.845020,0.050078,4.493726,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7296,9398,MTR_007,2008-12-31,1.431837,1.216527,0.053739,6.642979,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7297,10840,MTR_008,2008-12-31,1.484169,1.228541,0.094065,6.565422,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
7298,12282,MTR_009,2008-12-31,1.516974,1.254294,0.106465,6.716099,243.296618,2.967759,5.687639,25.8,13.8,0.3,19.3
